In [51]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json
import seaborn as sns
import results_functions
from scipy import stats
import pickle
from scipy.stats import wilcoxon


json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

### Make descriptive boxplots for all

In [ ]:
directory = os.path.join(json_path_onedrive,
    'results', 'Avg_Features')

reload(results_functions)
%matplotlib qt
saving = 1
df_fu0m, df_fu3m, df_fu12m = results_functions.get_descriptives(directory, saving)

'''
with open(os.path.join(directory,
    'Means_FU0M.pkl'), "rb") as file:
    val_dat = pickle.load(file)
'''

#### Pairwise Comparisons

In [ ]:
all_fus_df = pd.read_csv(os.path.join(
    json_path_onedrive, 'results', 'Avg_Features', 'All_FollowUp_dfs.csv'
)) 

all_fus_df.head()

In [ ]:
# Create a dictionary to store the results for each column
all_columns = ['Telemetry_AllMin', 'TelemDurSumSMinRes', 'TelemDurSumMinWard', 'SensDurSumMin']
wilcoxon_results = {}

# List of time points to compare
time_points = ['FU0M', 'FU3M', 'FU12M']

# Perform pairwise Wilcoxon signed-rank tests and store the results for each column
for column in all_columns:
    comparisons_results = {}
    for i in range(len(time_points)-1):
        for j in range(i+1, len(time_points)):
            tp1, tp2 = time_points[i], time_points[j]
            x1 = all_fus_df.loc[all_fus_df['TimePoint'] == tp1, column]
            x2 = all_fus_df.loc[all_fus_df['TimePoint'] == tp2, column]

            statistic, p_value = wilcoxon(x1, x2)

            comparison_name = f"{column}_{tp1}-{tp2}"
            significance = 'n.s.'  # Default value
            if p_value < 0.001:
                significance = '***'
            elif p_value < 0.01:
                significance = '**'
            elif p_value < 0.05:
                significance = '*'

            comparisons_results[comparison_name] = {'Statistic': statistic, 'p-value': p_value, 'Significance': significance}
    
    wilcoxon_results[column] = comparisons_results

# Convert the dictionary to a DataFrame
results_df = pd.DataFrame({(column, key): value for column, values in wilcoxon_results.items() for key, value in values.items()}).T

# Display the results DataFrame
results_df


In [ ]:
results_df.to_excel(os.path.join(
    json_path_onedrive, 'results', 'Avg_Features', 'test_results','PairwiseComps.xlsx'
))

In [ ]:
all_fus_df[(all_fus_df['TimePoint'] == 'FU0M') & (all_fus_df['Electrode'] == 'SenSight')]

### Make correlations with TEED

In [ ]:
directory_Feat = os.path.join(json_path_onedrive,
    'results', 'Avg_Features')

directory_TEED = os.path.join(json_path_onedrive,
    'results', 'Stim_pars', 'TEED')

directory_corrs = os.path.join(json_path_onedrive,
    'results', 'Correlations')

saving = 1

In [ ]:
reload(results_functions)

corr_df, correlation_stats = results_functions.get_battery_corrs(directory_Feat, 
                                                                 directory_TEED, 
                                                                 directory_corrs, 
                                                                 saving)



In [ ]:
correlation_stats

In [ ]:
check_df = pd.read_csv(os.path.join(
    json_path_onedrive, 'results', 'Avg_Features', 'Sub019_AvgFeatures.csv'
))

check_df

In [ ]:
with open(os.path.join(directory_corrs,
    'Stat_results.pkl'), "rb") as file:
    val_dat = pickle.load(file)
val_dat

In [ ]:
71077.0/3600